In [4]:
from Diffusion.similarity_models import TwoRandomWalksSampler
similarity_batch = TwoRandomWalksSampler(URM_train)
_ = similarity_batch.sample_warm_batch(500)
_

TypeError: __init__() missing 1 required positional argument: 'warm_user_ids'

In [1]:

from Data_manager.DataSplitter_Holdout import DataSplitter_Holdout
from Data_manager.Movielens.Movielens100KReader import Movielens100KReader
from Evaluation.Evaluator import EvaluatorHoldout
import numpy as np

dataset_reader = Movielens100KReader()

dataSplitter = DataSplitter_Holdout(dataset_reader, user_wise=False, split_interaction_quota_list=[80, 10, 10])
dataSplitter.load_data() #"results_experiments/Movielens1M/data"
URM_train, URM_validation, URM_test = dataSplitter.get_holdout_split()

cutoff_list = [10, 50]
evaluator_validation = EvaluatorHoldout(URM_validation, cutoff_list=cutoff_list)
evaluator_test = EvaluatorHoldout(URM_test, cutoff_list=cutoff_list)

DataSplitter_Holdout: Verifying data consistency...
DataSplitter_Holdout: Verifying data consistency... Passed!
DataSplitter_Holdout: DataReader: Movielens100K
	Num items: 1682
	Num users: 943
	Train 		quota 80.00 (80.00), 	interactions 80000, 	density 5.04E-02
	Validation 	quota 10.00 (10.00), 	interactions 10000, 	density 6.30E-03
	Test 		quota 10.00 (10.00), 	interactions 10000, 	density 6.30E-03





DataSplitter_Holdout: Done.
EvaluatorHoldout: Ignoring 27 ( 2.9%) Users that have less than 1 test interactions
EvaluatorHoldout: Ignoring 18 ( 1.9%) Users that have less than 1 test interactions


In [2]:
import optuna
import numpy as np
import pandas as pd
import os
from Diffusion.MultiBlockWSimilarityAttentionDiffusionRecommender import WSAD_Recommender

def objective(trial):

    cutoff = 10
    metric = 'NDCG'
    directory_path = '/Users/lucaortolomo/Desktop/TESI/Thesis_DiffusionRecommender-main/Self-Attention/OptunaResults/Movielens100K'

    batch_size = trial.suggest_categorical('batch_size', [64, 128, 256]) #512, 1024
    embeddings_dim = trial.suggest_categorical('embeddings_dim', [64, 128, 256]) #til 1024
    heads = trial.suggest_categorical('heads', [4])
    attention_blocks = trial.suggest_categorical('attention_blocks', [1, 2, 3]) # til 8 or more?
    d_ff = trial.suggest_categorical('d_ff', [512, 1024, 2048])
    epochs = trial.suggest_int('epochs', 50, 350)
    l2_reg = trial.suggest_loguniform('l2_reg', 1e-5, 1e-3)
    learning_rate = trial.suggest_loguniform('learning_rate', 1e-4, 1e-2)
    noise_timesteps = trial.suggest_int('noise_timesteps', 1, 700)
    inference_timesteps = trial.suggest_int('inference_timesteps', 1, noise_timesteps-1)
    start_beta = trial.suggest_float('start_beta', 0.00001, 0.001)
    end_beta = trial.suggest_float('end_beta', 0.01, 0.2)

    # Initialize and train the recommender

    diffusion_model = WSAD_Recommender(URM_train = URM_train, verbose = False, use_gpu = True)

    diffusion_model.fit(
                      epochs=epochs,
                      batch_size=batch_size,
                      embeddings_dim=embeddings_dim,
                      heads=heads,
                      attention_blocks = attention_blocks,
                      d_ff = d_ff,
                      l2_reg=l2_reg,
                      learning_rate=learning_rate,
                      noise_timesteps = noise_timesteps,
                      inference_timesteps = inference_timesteps,
                      start_beta = start_beta,
                      end_beta = end_beta
    )

    result_df, _ = evaluator_validation.evaluateRecommender(diffusion_model)
    hyperparams = {
    'batch_size': batch_size,
    'embeddings_dim': embeddings_dim,
    'heads': heads,
    'attention_blocks': attention_blocks,
    'd_ff': d_ff,
    'epochs': epochs,
    'l2_reg': l2_reg,
    'learning_rate': learning_rate,
    'noise_timesteps': noise_timesteps,
    'inference_timesteps': inference_timesteps,
    'start_beta': start_beta,
    'end_beta': end_beta}

    result_df['hyperparams'] = str(hyperparams)

    filename = directory_path + diffusion_model.RECOMMENDER_NAME + ".csv"

    # Check if file exists
    if os.path.isfile(filename):
        # If it exists, append without writing the header
        pd.DataFrame(result_df.loc[cutoff]).transpose().to_csv(filename, mode='a', header=False, index=False)
    else:
        # If it doesn't exist, create it, write the header
        pd.DataFrame(result_df.loc[cutoff]).transpose().to_csv(filename, mode='w', header=True, index=False)

    return result_df.loc[cutoff][metric]



In [2]:
import optuna
import numpy as np
import pandas as pd
import os
from Diffusion.MultiBlockSimilarityAttentionDiffusionRecommender import MultiBlockSimilarityAttentionDiffusionRecommender

def objective(trial):

    cutoff = 10
    metric = 'NDCG'
    directory_path = '/Users/lucaortolomo/Desktop/TESI/Thesis_DiffusionRecommender-main/Self-Attention/OptunaResults/Movielens100K'

    batch_size = trial.suggest_categorical('batch_size', [64, 128, 256, 512]) # , 1024]) # Movielens100k has only 943 users!!
    embeddings_dim = trial.suggest_categorical('embeddings_dim', [64, 128, 256, 512, 1024])
    heads = trial.suggest_categorical('heads', [1, 4, 8, 16])
    attention_blocks = trial.suggest_categorical('attention_blocks', [1, 2, 3])
    d_ff = trial.suggest_categorical('d_ff', [1024, 2048, 4096])
    epochs = trial.suggest_int('epochs', 50, 750)
    l2_reg = trial.suggest_loguniform('l2_reg', 1e-5, 1e-3)
    learning_rate = trial.suggest_loguniform('learning_rate', 1e-4, 1e-2)
    noise_timesteps = trial.suggest_int('noise_timesteps', 3, 1000)
    inference_timesteps = trial.suggest_int('inference_timesteps', 1, noise_timesteps-1)
    start_beta = trial.suggest_float('start_beta', 0.00001, 0.001)
    end_beta = trial.suggest_float('end_beta', 0.01, 0.2)
    similarity_weight = trial.suggest_float('similiraty_weight', 0.1, 0.2)


    # Initialize and train the recommender

    diffusion_model = MultiBlockSimilarityAttentionDiffusionRecommender(URM_train = URM_train, verbose = False, use_gpu = True)

    diffusion_model.fit(
                      epochs=epochs,
                      batch_size=batch_size,
                      embeddings_dim=embeddings_dim,
                      heads=heads,
                      attention_blocks = attention_blocks,
                      d_ff = d_ff,
                      l2_reg=l2_reg,
                      learning_rate=learning_rate,
                      noise_timesteps = noise_timesteps,
                      inference_timesteps = inference_timesteps,
                      start_beta = start_beta,
                      end_beta = end_beta
    )

    result_df, _ = evaluator_validation.evaluateRecommender(diffusion_model)
    hyperparams = {
    'batch_size': batch_size,
    'embeddings_dim': embeddings_dim,
    'heads': heads,
    'attention_blocks': attention_blocks,
    'd_ff': d_ff,
    'epochs': epochs,
    'l2_reg': l2_reg,
    'learning_rate': learning_rate,
    'noise_timesteps': noise_timesteps,
    'inference_timesteps': inference_timesteps,
    'start_beta': start_beta,
    'end_beta': end_beta}

    result_df['hyperparams'] = str(hyperparams)

    filename = directory_path + diffusion_model.RECOMMENDER_NAME + ".csv"

    # Check if file exists
    if os.path.isfile(filename):
        # If it exists, append without writing the header
        pd.DataFrame(result_df.loc[cutoff]).transpose().to_csv(filename, mode='a', header=False, index=False)
    else:
        # If it doesn't exist, create it, write the header
        pd.DataFrame(result_df.loc[cutoff]).transpose().to_csv(filename, mode='w', header=True, index=False)

    return result_df.loc[cutoff][metric]


In [3]:
# <class 'numpy.ndarray'>
# (916, 1682)
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=50,show_progress_bar=True)

[I 2024-04-01 12:58:59,333] A new study created in memory with name: no-name-42f426ad-77a8-4a44-ab36-dc87a223f919


  0%|          | 0/50 [00:00<?, ?it/s]

/var/folders/vy/ssvybtks5ms1n3l5ldrh_hv40000gn/T/ipykernel_43564/3566311502.py:19: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  l2_reg = trial.suggest_loguniform('l2_reg', 1e-5, 1e-3)
/var/folders/vy/ssvybtks5ms1n3l5ldrh_hv40000gn/T/ipykernel_43564/3566311502.py:20: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  learning_rate = trial.suggest_loguniform('learning_rate', 1e-4, 1e-2)
/Users/lucaortolomo/miniconda3/envs/Tesi/lib/python3.8/site-packages/scipy/sparse/_index.py:137: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  self._set_arrayXarray_sparse(i, j, x)


SAD: Epoch 1 of 204. Elapsed time 0.92 sec
SAD: Epoch 2 of 204. Elapsed time 0.98 sec
SAD: Epoch 3 of 204. Elapsed time 1.04 sec
SAD: Epoch 4 of 204. Elapsed time 1.09 sec
SAD: Epoch 5 of 204. Elapsed time 1.15 sec
SAD: Epoch 6 of 204. Elapsed time 1.21 sec
SAD: Epoch 7 of 204. Elapsed time 1.26 sec
SAD: Epoch 8 of 204. Elapsed time 1.32 sec
SAD: Epoch 9 of 204. Elapsed time 1.38 sec
SAD: Epoch 10 of 204. Elapsed time 1.43 sec
SAD: Epoch 11 of 204. Elapsed time 1.49 sec
SAD: Epoch 12 of 204. Elapsed time 1.55 sec
SAD: Epoch 13 of 204. Elapsed time 1.60 sec
SAD: Epoch 14 of 204. Elapsed time 1.66 sec
SAD: Epoch 15 of 204. Elapsed time 1.71 sec
SAD: Epoch 16 of 204. Elapsed time 1.77 sec
SAD: Epoch 17 of 204. Elapsed time 1.83 sec
SAD: Epoch 18 of 204. Elapsed time 1.89 sec
SAD: Epoch 19 of 204. Elapsed time 1.95 sec
SAD: Epoch 20 of 204. Elapsed time 2.01 sec
SAD: Epoch 21 of 204. Elapsed time 2.07 sec
SAD: Epoch 22 of 204. Elapsed time 2.12 sec
SAD: Epoch 23 of 204. Elapsed time 2.18 s

/var/folders/vy/ssvybtks5ms1n3l5ldrh_hv40000gn/T/ipykernel_43564/3566311502.py:19: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  l2_reg = trial.suggest_loguniform('l2_reg', 1e-5, 1e-3)
/var/folders/vy/ssvybtks5ms1n3l5ldrh_hv40000gn/T/ipykernel_43564/3566311502.py:20: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  learning_rate = trial.suggest_loguniform('learning_rate', 1e-4, 1e-2)
/Users/lucaortolomo/miniconda3/envs/Tesi/lib/python3.8/site-packages/scipy/sparse/_index.py:137: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  self._set_arrayXarray_sparse(i, j, x)


...

SAD: Epoch 1 of 120. Elapsed time 0.28 sec
SAD: Epoch 2 of 120. Elapsed time 0.55 sec
SAD: Epoch 3 of 120. Elapsed time 0.81 sec
SAD: Epoch 4 of 120. Elapsed time 1.08 sec
SAD: Epoch 5 of 120. Elapsed time 1.35 sec
SAD: Epoch 6 of 120. Elapsed time 1.61 sec
SAD: Epoch 7 of 120. Elapsed time 1.88 sec
SAD: Epoch 8 of 120. Elapsed time 2.15 sec
SAD: Epoch 9 of 120. Elapsed time 2.41 sec
SAD: Epoch 10 of 120. Elapsed time 2.68 sec
SAD: Epoch 11 of 120. Elapsed time 2.95 sec
SAD: Epoch 12 of 120. Elapsed time 3.21 sec
SAD: Epoch 13 of 120. Elapsed time 3.48 sec
SAD: Epoch 14 of 120. Elapsed time 3.75 sec
SAD: Epoch 15 of 120. Elapsed time 4.01 sec
SAD: Epoch 16 of 120. Elapsed time 4.28 sec
SAD: Epoch 17 of 120. Elapsed time 4.54 sec
SAD: Epoch 18 of 120. Elapsed time 4.81 sec
SAD: Epoch 19 of 120. Elapsed time 5.08 sec
SAD: Epoch 20 of 120. Elapsed time 5.35 sec
SAD: Epoch 21 of 120. Elapsed time 5.61 sec
SAD: Epoch 22 of 120. Elapsed time 5.88 sec
SAD: Epoch 23 of 120. Elapsed time 6.15 s

/var/folders/vy/ssvybtks5ms1n3l5ldrh_hv40000gn/T/ipykernel_43564/3566311502.py:19: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  l2_reg = trial.suggest_loguniform('l2_reg', 1e-5, 1e-3)
/var/folders/vy/ssvybtks5ms1n3l5ldrh_hv40000gn/T/ipykernel_43564/3566311502.py:20: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  learning_rate = trial.suggest_loguniform('learning_rate', 1e-4, 1e-2)
/Users/lucaortolomo/miniconda3/envs/Tesi/lib/python3.8/site-packages/scipy/sparse/_index.py:137: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  self._set_arrayXarray_sparse(i, j, x)


SAD: Epoch 1 of 86. Elapsed time 0.29 sec
SAD: Epoch 2 of 86. Elapsed time 0.56 sec
SAD: Epoch 3 of 86. Elapsed time 0.84 sec
SAD: Epoch 4 of 86. Elapsed time 1.10 sec
SAD: Epoch 5 of 86. Elapsed time 1.37 sec
SAD: Epoch 6 of 86. Elapsed time 1.64 sec
SAD: Epoch 7 of 86. Elapsed time 1.91 sec
SAD: Epoch 8 of 86. Elapsed time 2.19 sec
SAD: Epoch 9 of 86. Elapsed time 2.47 sec
SAD: Epoch 10 of 86. Elapsed time 2.74 sec
SAD: Epoch 11 of 86. Elapsed time 3.02 sec
SAD: Epoch 12 of 86. Elapsed time 3.29 sec
SAD: Epoch 13 of 86. Elapsed time 3.56 sec
SAD: Epoch 14 of 86. Elapsed time 3.83 sec
SAD: Epoch 15 of 86. Elapsed time 4.10 sec
SAD: Epoch 16 of 86. Elapsed time 4.38 sec
SAD: Epoch 17 of 86. Elapsed time 4.64 sec
SAD: Epoch 18 of 86. Elapsed time 4.92 sec
SAD: Epoch 19 of 86. Elapsed time 5.19 sec
SAD: Epoch 20 of 86. Elapsed time 5.47 sec
SAD: Epoch 21 of 86. Elapsed time 5.74 sec
SAD: Epoch 22 of 86. Elapsed time 6.01 sec
SAD: Epoch 23 of 86. Elapsed time 6.28 sec
SAD: Epoch 24 of 86.

/var/folders/vy/ssvybtks5ms1n3l5ldrh_hv40000gn/T/ipykernel_43564/3566311502.py:19: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  l2_reg = trial.suggest_loguniform('l2_reg', 1e-5, 1e-3)
/var/folders/vy/ssvybtks5ms1n3l5ldrh_hv40000gn/T/ipykernel_43564/3566311502.py:20: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  learning_rate = trial.suggest_loguniform('learning_rate', 1e-4, 1e-2)
/Users/lucaortolomo/miniconda3/envs/Tesi/lib/python3.8/site-packages/scipy/sparse/_index.py:137: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  self._set_arrayXarray_sparse(i, j, x)


SAD: Epoch 1 of 88. Elapsed time 0.28 sec
SAD: Epoch 2 of 88. Elapsed time 0.55 sec
SAD: Epoch 3 of 88. Elapsed time 0.83 sec
SAD: Epoch 4 of 88. Elapsed time 1.10 sec
SAD: Epoch 5 of 88. Elapsed time 1.37 sec
SAD: Epoch 6 of 88. Elapsed time 1.65 sec
SAD: Epoch 7 of 88. Elapsed time 1.93 sec
SAD: Epoch 8 of 88. Elapsed time 2.20 sec
SAD: Epoch 9 of 88. Elapsed time 2.47 sec
SAD: Epoch 10 of 88. Elapsed time 2.75 sec
SAD: Epoch 11 of 88. Elapsed time 3.02 sec
SAD: Epoch 12 of 88. Elapsed time 3.29 sec
SAD: Epoch 13 of 88. Elapsed time 3.57 sec
SAD: Epoch 14 of 88. Elapsed time 3.84 sec
SAD: Epoch 15 of 88. Elapsed time 4.11 sec
SAD: Epoch 16 of 88. Elapsed time 4.38 sec
SAD: Epoch 17 of 88. Elapsed time 4.65 sec
SAD: Epoch 18 of 88. Elapsed time 4.92 sec
SAD: Epoch 19 of 88. Elapsed time 5.20 sec
SAD: Epoch 20 of 88. Elapsed time 5.47 sec
SAD: Epoch 21 of 88. Elapsed time 5.74 sec
SAD: Epoch 22 of 88. Elapsed time 6.01 sec
SAD: Epoch 23 of 88. Elapsed time 6.28 sec
SAD: Epoch 24 of 88.

In [ ]:
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=200,show_progress_bar=True)

In [10]:
from Diffusion.DiffusionRecommender import MultiBlockAttentionDiffusionRecommender
diffusion_model = MultiBlockAttentionDiffusionRecommender(URM_train = URM_train, use_gpu = True)
diffusion_model.fit(epochs=5)

MultiBlockAttentionDiffusionRecommender: URM Detected 26 ( 1.5%) items with no interactions.
MultiBlockAttentionDiffusionRecommender: Epoch 1, loss 6.97E+00
MultiBlockAttentionDiffusionRecommender: Epoch 1 of 5. Elapsed time 4.19 sec
MultiBlockAttentionDiffusionRecommender: Epoch 2, loss 6.17E+00
MultiBlockAttentionDiffusionRecommender: Epoch 2 of 5. Elapsed time 4.38 sec
MultiBlockAttentionDiffusionRecommender: Epoch 3, loss 6.28E+00
MultiBlockAttentionDiffusionRecommender: Epoch 3 of 5. Elapsed time 4.57 sec
MultiBlockAttentionDiffusionRecommender: Epoch 4, loss 6.13E+00
MultiBlockAttentionDiffusionRecommender: Epoch 4 of 5. Elapsed time 4.75 sec
MultiBlockAttentionDiffusionRecommender: Epoch 5, loss 6.20E+00
MultiBlockAttentionDiffusionRecommender: Epoch 5 of 5. Elapsed time 4.93 sec
MultiBlockAttentionDiffusionRecommender: Terminating at epoch 5. Elapsed time 4.94 sec
MultiBlockAttentionDiffusionRecommender: Training complete


In [11]:
result_df, _ = evaluator_validation.evaluateRecommender(diffusion_model)
result_df

EvaluatorHoldout: Processed 916 (100.0%) in 2.60 sec. Users per second: 353


,PRECISION,PRECISION_RECALL_MIN_DEN,RECALL,MAP,MAP_MIN_DEN,MRR,NDCG,F1,HIT_RATE,ARHR_ALL_HITS,...,COVERAGE_USER,COVERAGE_USER_HIT,USERS_IN_GT,DIVERSITY_GINI,SHANNON_ENTROPY,RATIO_DIVERSITY_HERFINDAHL,RATIO_DIVERSITY_GINI,RATIO_SHANNON_ENTROPY,RATIO_AVERAGE_POPULARITY,RATIO_NOVELTY
cutoff,,,,,,,,,,,,,,,,,,,,,
10,0.095306,0.141,0.109041,0.042878,0.062498,0.258577,0.117581,0.101712,0.56441,0.330026,...,0.971368,0.54825,0.971368,0.011682,4.558766,0.952265,0.031481,0.469742,2.290069,0.095436
50,0.056659,0.276977,0.276483,0.01491,0.066893,0.271178,0.180707,0.094046,0.796943,0.411937,...,0.971368,0.774125,0.971368,0.045353,6.563823,0.989559,0.122222,0.676347,1.652778,0.507614


In [3]:

from Data_manager.DataSplitter_Holdout import DataSplitter_Holdout
from Data_manager.CiteULike.CiteULikeReader import CiteULike_aReader
from Evaluation.Evaluator import EvaluatorHoldout
import numpy as np

dataset_reader = CiteULike_aReader()

dataSplitter = DataSplitter_Holdout(dataset_reader, user_wise=False, split_interaction_quota_list=[80, 10, 10])
dataSplitter.load_data('/Users/lucaortolomo/Desktop/TESI/Thesis_DiffusionRecommender-main/Hyperparameter_databases/hyperparameter_database_2024_02/k_5_cores/original/hyperopt_random_holdout_80_10_10/CiteULike_a/data') #"results_experiments/Movielens1M/data"
URM_train, URM_validation, URM_test = dataSplitter.get_holdout_split()

cutoff_list = [10, 50]
evaluator_validation = EvaluatorHoldout(URM_validation, cutoff_list=cutoff_list)
evaluator_test = EvaluatorHoldout(URM_test, cutoff_list=cutoff_list)

DataSplitter_Holdout: Verifying data consistency...
DataSplitter_Holdout: Verifying data consistency... Passed!
DataSplitter_Holdout: DataReader: CiteULike_a
	Num items: 15429
	Num users: 5536
	Train 		quota 80.00 (80.00), 	interactions 160144, 	density 1.87E-03
	Validation 	quota 10.00 (10.00), 	interactions 20018, 	density 2.34E-04
	Test 		quota 10.00 (10.00), 	interactions 20018, 	density 2.34E-04



	ICM name: ICM_title_abstract, Num features: 7999, feature occurrences: 1031068, density 8.35E-03


DataSplitter_Holdout: Done.
EvaluatorHoldout: Ignoring 795 (14.4%) Users that have less than 1 test interactions
EvaluatorHoldout: Ignoring 742 (13.4%) Users that have less than 1 test interactions


In [5]:
import os
directory_path = '/Users/lucaortolomo/Desktop/TESI/Thesis_DiffusionRecommender-main/Self-Attention/OptunaResults/Dataset/' + "full" + '/' 
if not os.path.exists(directory_path):
    os.makedirs(directory_path)
    print(f"Directory {directory_path} created.")

Directory /Users/lucaortolomo/Desktop/TESI/Thesis_DiffusionRecommender-main/Self-Attention/OptunaResults/Dataset/full/ created.


In [9]:
import torch
from Diffusion.similarity_models import TwoRandomWalksSampler

user = 200
sampler = TwoRandomWalksSampler(URM=URM_train, warm_user_ids=None)
user_batch = sampler.sample_batch(256, user)
user_batch_tensor = URM_train[user_batch]
user_profile_reference = URM_train[user].toarray()

# Convert CSR matrix to a dense numpy array directly
user_batch_dense_np = user_batch_tensor.toarray()

# Convert the dense numpy array to a PyTorch tensor
# and move it to the appropriate device
if str('mps') == 'mps':
    user_batch_tensor = torch.tensor(user_batch_dense_np, dtype=torch.float32, device='cpu').to('mps')
else:
# Transferring only the sparse structure to reduce the data transfer
    user_batch_tensor = torch.sparse_csr_tensor(user_batch_tensor.indptr,
                                                user_batch_tensor.indices,
                                                user_batch_tensor.data,
                                                size=user_batch_tensor.shape,
                                                dtype=torch.float32,
                                                device='cpu',
                                                requires_grad=False).to_dense()
    
print(user_batch_tensor)

/Users/lucaortolomo/miniconda3/envs/Tesi/lib/python3.8/site-packages/scipy/sparse/_index.py:137: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  self._set_arrayXarray_sparse(i, j, x)


tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [3., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]], device='mps:0')


[[0. 0. 0. ... 0. 0. 0.]]
